In [3]:
import pandas as pd
import numpy as np

In [4]:
#  HERE WE LOAD THE THE CSV FILE OF AMAZON IN  SALES DATA OF YEAR 20222   
df=pd.read_csv('1aammazon.csv')
 
# THIS COMMAND DROP ALL THE  ROWS WITH NULL VALES , 
df.dropna()

# NOW I REMOVE THE UNECCESSARY SPACE AND CHARACTERS IN COLUMN NAME FOR EASY TO USE OR TYPE 
df.columns= df.columns.str.lower().str.strip().str.replace(' ','_').str.replace('-','_')

# HERE WE  SET THE COLUMN DATE WITH DATE_TIME_INDEX IN PANDAS  FOR EASY TO ACCES DATE ,MONTH , YEAR ONLY  
df.date=pd.to_datetime(df.date)

# NOW WE CHECK DATA FERAME 
df.head(2)

,index,order_id,date,status,fulfilment,sales_channel,ship_service_level,style,sku,category,...,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,promotion_ids,b2b,fulfilled_by,unnamed:_22
0,43725,402-7472994-8693158,2022-04-04,Shipped,Amazon,Amazon.in,Expedited,SET271,SET271-KR-NP-XS,Set,...,INR,885.0,FERRARGUNJ,ANDAMAN & NICOBAR,744103.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN
1,37064,403-8616092-1924316,2022-04-08,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3797,JNE3797-KR-L,Western Dress,...,INR,725.0,FERRARGUNJ,ANDAMAN & NICOBAR,744103.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN


#  (Q1)=> Top ten state with max order count

In [113]:
df.groupby('ship_state').order_id.count().reset_index(name='no of oerder')\
.sort_values(by='no of oerder',ascending=False).head(10)

,ship_state,no of oerder
26,MAHARASHTRA,10417
21,KARNATAKA,8256
44,TELANGANA,5449
43,TAMIL NADU,5230
46,UTTAR PRADESH,4864
12,DELHI,3322
22,KERALA,2687
48,WEST BENGAL,2634
1,ANDHRA PRADESH,2326
17,HARYANA,2069


# (Q2)=> show total number of order count total amount with max . min amount

In [114]:
df.groupby('category').agg(max_amount=('amount','max'),\
min_amount=('amount','min'),sum_amount=('amount','sum'),order_no=('order_id','count'))

,max_amount,min_amount,sum_amount,order_no
category,,,,
Blouse,1266.66,0.0,173003.77,353
Bottom,1028.58,0.0,48863.89,147
Dupatta,305.00,305.0,305.00,1
Ethnic Dress,1449.00,0.0,308224.80,443
Saree,1248.00,0.0,35048.19,50
Set,5495.00,0.0,17975916.77,23197
Top,1797.00,0.0,2361257.02,4712
Western Dress,2655.00,0.0,4937370.08,6858
kurta,2796.00,0.0,9759005.91,22920


# (Q3)=> FOR EVERY MONTH AND STATE SHOW TOP 2 MAX AMOUNT ORDER DETAIL WHERE  PRODUCT CATEGORY IS NOT "SET"

In [115]:
dfQ=df[df.category!='Set']
dfQ.groupby([df.date.dt.month_name(),'ship_state'])\
.apply(lambda x: x.nlargest(2,columns=['amount']))\
[['date','category','order_id','ship_city','amount']].rename(columns={'date':'datee'},level=0)\
.reset_index(level=2,drop=True).reset_index()

,date,ship_state,datee,category,order_id,ship_city,amount
0,April,ANDAMAN & NICOBAR,2022-04-03,Western Dress,171-0807078-4971518,PORT BLAIR,1033.00
1,April,ANDAMAN & NICOBAR,2022-04-23,Western Dress,402-1396331-3893925,PORTBLAIR,856.19
2,April,ANDHRA PRADESH,2022-04-01,Ethnic Dress,171-5793625-8052350,KAKINADA,1174.00
3,April,ANDHRA PRADESH,2022-04-01,Western Dress,405-2258415-2444323,VIJAYAWADA,1099.00
4,April,ARUNACHAL PRADESH,2022-04-20,Western Dress,403-4321280-9677103,Itanagar,859.00
...,...,...,...,...,...,...,...
252,May,UTTARAKHAND,2022-05-30,Western Dress,171-1868964-1977150,DEHRADUN,885.00
253,May,UTTARAKHAND,2022-05-19,kurta,404-7580978-4249953,KASHIPUR,869.00
254,May,WEST BENGAL,2022-05-20,kurta,406-3078152-5286722,HOWRAH,1099.00
255,May,WEST BENGAL,2022-05-25,kurta,406-6800199-7385123,KOLKATA,1099.00


# (Q4)=> FOR EACH MONTH SHOW  TOP 2 MOST FAMOUS PRODUCT CATEGORY IN TERM {{AMOUNT}}

In [2]:
df.groupby([df.date.dt.to_period('M'),'category']).apply(lambda x: x.amount.sum())\
.reset_index(name='amount').groupby('date').apply(lambda x: x.nlargest(3,'amount'))\
.reset_index(drop=True).head()

NameError: name 'df' is not defined

# (Q5)=> FOR EACH STATE COUNT NUMBER OF ORDER 

In [124]:
df.groupby('ship_state').order_id.count().reset_index().sort_values('order_id',ascending=False).head()


,ship_state,order_id
26,MAHARASHTRA,10417
21,KARNATAKA,8256
44,TELANGANA,5449
43,TAMIL NADU,5230
46,UTTAR PRADESH,4864


# (Q6)=> FOR THIS  ['MAHARASHTRA','DELHI','ANDHRA PRADESH'] STATE  FIND GROWTH PERCENTAGE OF **ORDER COUNT**  FROM MONTH  APRIL TO MAY 
# USE OF PIVOT TABLE AND 

In [118]:
df[(df.ship_state.isin(['MAHARASHTRA','DELHI','ANDHRA PRADESH']))&(df.date.dt.month.isin([4,5]))]\
.groupby([df.date.dt.strftime('%Y-%B'),'ship_state']).order_id.count().reset_index(name='order_in_maharash')\
.pivot_table(index='ship_state',values='order_in_maharash',columns='date').reset_index().rename_axis(columns=None)\
.assign(growth=lambda x: ((x['2022-May']-x['2022-April'])/x['2022-April']*100).round(2).astype(str)+'%')

,ship_state,2022-April,2022-May,growth
0,ANDHRA PRADESH,839,745,-11.2%
1,DELHI,1310,1080,-17.56%
2,MAHARASHTRA,3982,3416,-14.21%


# (Q7)=> ANOMALY FIND
# STATE WITH  MINIMUM ORDER BUT  MAX AMOUNT

In [119]:
df.groupby('ship_state').agg(order_count=('order_id','count'),total_amount=('amount','sum'))\
.assign(t= lambda x: x.total_amount/x.order_count).nlargest(3,'t').iloc[:,:-1]

,order_count,total_amount
ship_state,,
Punjab,3,2932.00
LADAKH,20,18320.43
Chandigarh,4,3270.62


# (Q8)=> ANOMALY FIND
# STATE WITH  MINIMUM AMOUNT BUT  MAX ORDRER

In [120]:
df.groupby('ship_state').agg(order_count=('order_id','count'),total_amount=('amount','sum'))\
.assign(t= lambda x: x.total_amount/x.order_count).nsmallest(3,'t').iloc[:,:-1]

,order_count,total_amount
ship_state,,
Arunachal pradesh,1,301.0
Goa,6,1937.0
Manipur,1,339.0


# (Q9)=> CITY WITH MAX ORDRE COUNT WHERE FULFILMENT BY  AMAZON IN JULY MONTH

In [122]:
df[df.fulfilment.isin(['Amazon'])&df.date.dt.month.isin([5])].groupby(['ship_state','ship_city'])\
.order_id.count().reset_index(name='order_count').nlargest(1,'order_count')


,ship_state,ship_city,order_count
434,KARNATAKA,BENGALURU,1269


# (Q10)=> TOP 2 MONTH WITH HIGHEST AMOUNT

In [121]:
df.groupby(df.date.dt.strftime('%Y-%b')).amount.sum()\
.reset_index(name='total_amount').nlargest(2,'total_amount')

,date,total_amount
0,2022-Apr,13176537.61
3,2022-May,11816880.93


# (Q11)=> USE OF   TABLE JOIN 
# STATE WHERE 50% OF PRODUCT IS BY MERCHANTE

In [123]:
df1 =df.groupby('ship_state').order_id.count().reset_index(name='total_order')
df2=df[df.fulfilment=='Amazon'].groupby('ship_state').order_id.count().reset_index(name='total_order_from_amazon')
df3=pd.merge(df1,df2,on='ship_state')
df3[(df3.total_order/2)<df3.total_order_from_amazon].head()

,ship_state,total_order,total_order_from_amazon
0,ANDAMAN & NICOBAR,116,73
1,ANDHRA PRADESH,2326,1597
2,ARUNACHAL PRADESH,54,33
3,ASSAM,709,462
4,Arunachal Pradesh,1,1
